In [1]:
# coding: utf-8
import argparse
import glob
import librosa
from tensorflow.keras.layers import BatchNormalization, Multiply, Add
from tensorflow.keras.layers import Conv1D
from tensorflow. keras.utils import multi_gpu_model
import tensorflow.keras.callbacks
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import GRU
from tensorflow.keras.models import Model
from tensorflow.keras.layers import add, concatenate
from tensorflow.keras.layers import Reshape, Lambda
from tensorflow.keras.layers import Input, Dense, Activation
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv1D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.callbacks import CSVLogger, ReduceLROnPlateau
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow.keras.backend as K
import datetime
import re
import itertools
import platform
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from sklearn.metrics import accuracy_score
import pickle
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import pandas as pd
from tqdm import tqdm
import os
import codecs
import numpy as np
from os.path import join
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
tensorflow.compat.v1.keras.backend.set_session(sess)
tensorflow.compat.v1.keras.backend.clear_session()  # 清理session

np.random.seed(2018)



Instructions for updating:
non-resource variables are not supported in the long term



In [7]:
y_true = [[0,1,2],[2,1,0]]
y_pred = [[[1,0,0],[0,1,0],[0,0,1]], [[0,0,1],[0,0.3,0.7],[1,0,0]]]
input_length = [[3],[3]]
label_length = [[3],[3]]
K.ctc_batch_cost(y_true, y_pred, input_length, label_length)

<tf.Tensor 'ExpandDims_5:0' shape=(2, 1) dtype=float32>

In [9]:
input_tensor = Input(shape=(680,26), name="the_input")
input_tensor

<tf.Tensor 'the_input_1:0' shape=(?, 680, 26) dtype=float32>

In [12]:
x = Conv1D(kernel_size=1, filters=192, padding="same")(input_tensor)
x

<tf.Tensor 'conv1d_2/BiasAdd:0' shape=(?, 680, 192) dtype=float32>

In [15]:
x = BatchNormalization(axis=-1)(x)
x = Activation("tanh")(x)
x

<tf.Tensor 'activation_1/Tanh:0' shape=(?, 680, 192) dtype=float32>

In [16]:
def res_block(x, size, rate, dim=192):
        x_tanh = Conv1D(kernel_size=size, filters=dim,
                        dilation_rate=rate, padding="same")(x)
        x_tanh = BatchNormalization(axis=-1)(x_tanh)
        x_tanh = Activation("tanh")(x_tanh)
        x_sigmoid = Conv1D(kernel_size=size, filters=dim,
                           dilation_rate=rate, padding="same")(x)
        x_sigmoid = BatchNormalization(axis=-1)(x_sigmoid)
        x_sigmoid = Activation("sigmoid")(x_sigmoid)
        out = Multiply()([x_tanh, x_sigmoid])
        out = Conv1D(kernel_size=1, filters=dim, padding="same")(out)
        out = BatchNormalization(axis=-1)(out)
        out = Activation("tanh")(out)
        x = Add()([x, out])
        return x, out

In [18]:
skip = []
for i in np.arange(0, 2):
    for r in [1, 2, 4, 8, 16]:
        x, s = res_block(x, size=7, rate=r)
        skip.append(s)
skip

[<tf.Tensor 'activation_34/Tanh:0' shape=(?, 680, 192) dtype=float32>,
 <tf.Tensor 'activation_37/Tanh:0' shape=(?, 680, 192) dtype=float32>,
 <tf.Tensor 'activation_40/Tanh:0' shape=(?, 680, 192) dtype=float32>,
 <tf.Tensor 'activation_43/Tanh:0' shape=(?, 680, 192) dtype=float32>,
 <tf.Tensor 'activation_46/Tanh:0' shape=(?, 680, 192) dtype=float32>,
 <tf.Tensor 'activation_49/Tanh:0' shape=(?, 680, 192) dtype=float32>,
 <tf.Tensor 'activation_52/Tanh:0' shape=(?, 680, 192) dtype=float32>,
 <tf.Tensor 'activation_55/Tanh:0' shape=(?, 680, 192) dtype=float32>,
 <tf.Tensor 'activation_58/Tanh:0' shape=(?, 680, 192) dtype=float32>,
 <tf.Tensor 'activation_61/Tanh:0' shape=(?, 680, 192) dtype=float32>]

In [19]:
output_size = 1000
skip_tensor = Add()([s for s in skip])
logit = Conv1D(kernel_size=1, filters=192, padding="same")(skip_tensor)
logit = BatchNormalization(axis=-1)(logit)
logit = Activation("tanh")(logit)
y_pred = Conv1D(kernel_size=1, filters=output_size,
                padding="same", activation="softmax")(logit)
y_pred

<tf.Tensor 'conv1d_64/truediv:0' shape=(?, 680, 1000) dtype=float32>